# Solution for Probability

_This challenge was inspired by an actual recruitment challenge, where you program a bot to play continuous blackjack against other bots. I've always wondered if it used the python `random` class, so this challenge is a natural result of that train of thought, plus the question "how big can I win?". Since it also requires knowledge of probability and dynamic programming, I had intended this to be the hardest challenge in the category, so was pleasantly surprised by the number of solves._

In [1]:
from pwn import *
from z3 import *
import random

We use z3 to solve the MT19937 state for us. Given 624 consecutive outputs of `random.random()`, we can predict all future outputs. See for example, the [symbolic mersenne cracker](https://github.com/icemonster/symbolic_mersenne_cracker/), but we will use a faster but more concise implementation here:

In [2]:
def mtcrack_floats(arr):
    MT = [BitVec(f'm{i}', 32) for i in range(624)]
    s = Solver()
    
    def cache(x): # for some reason, this makes it faster in certain places
        tmp = Const(f'c{len(s.assertions())}', x.sort())
        s.add(tmp == x)
        return tmp
    
    def tamper(y):
        y ^= LShR(y, 11)
        y = cache(y ^ (y << 7) & 0x9D2C5680)
        y ^= cache((y << 15) & 0xEFC60000)
        return y ^ LShR(y, 18)
        
    def getnext():
        x = Concat(Extract(31, 31, MT[0]), Extract(30, 0, MT[1]))
        y = If(x & 1 == 0, BitVecVal(0, 32), 0x9908B0DF)
        MT.append(MT[397] ^ LShR(x, 1) ^ y)
        return tamper(MT.pop(0))
        
    def getrandbits(n):
        return Extract(31, 32 - n, getnext())
        
    s.add([Concat(getrandbits(27), getrandbits(26)) == n * 2**53 for n in arr])
    assert(s.check() == sat)
    state = [s.model().eval(x).as_long() for x in MT]
    
    r = random.Random()
    r.setstate((3, tuple(state + [0]), None))
    return r

We are ready to begin! Let's just give names to some of these constants -- we need to win 800 of 1337 games.

In [3]:
TOTAL_ROUNDS = 1337
WIN_THRESHOLD = 800
sh = remote('fun.chall.seetf.sg', 30001)

[x] Opening connection to fun.chall.seetf.sg on port 30001
[x] Opening connection to fun.chall.seetf.sg on port 30001: Trying 34.131.197.225
[+] Opening connection to fun.chall.seetf.sg on port 30001: Done


The strategy here is to split the game into two parts.
- Part 1 involves getting enough random numbers to get the MT19937 state.
- Part 2 uses the known state to play optimally and win as much as possible.

## Part 1

We know that we need at least 624 floats to crack the state. We could just hit or stand randomly, but this would be quite a waste of these early rounds (800 of 1337 is quite a lot). Using some actual probability (see Appendix below), we can use the strategy of hitting until our sum exceeds 0.5705565, and then standing. This will on average win us 77 of the first 181 rounds, at which point we will have enough floats to get our random state.


In [4]:
draws = []
def draw():
    sh.recvuntil(b'draw a [')
    n = float(sh.recvuntil(b']', True))
    draws.append(n)
    return n

r1rounds = 0
r1wins = 0

while True:
    total = draw()
    if (len(draws) >= 624):
        break

    PART1_THRESHOLD = 0.57055652829519647683
    while total < PART1_THRESHOLD:
        sh.sendline(b'h')
        total += draw()
       
    r1rounds += 1
    if total < 1:
        sh.sendline(b's')
        
    # collect dealer's numbers
    dealer = [float(n) for n in re.findall(b'\[(.+)\]', sh.recvuntil(b'Round'))]
    if total < 1 and sum(dealer) >= 1:
        r1wins += 1
        
    draws += dealer

r2rounds = TOTAL_ROUNDS - r1rounds
print(f'I won {r1wins} of the first {r1rounds} rounds ({r1wins*100/r1rounds:.1f}%)')
print(f'I need to win {WIN_THRESHOLD-r1wins} of the remaining {TOTAL_ROUNDS-r1rounds} rounds ({(WIN_THRESHOLD-r1wins)*100/(TOTAL_ROUNDS-r1rounds):.1f}%)')

I won 84 of the first 183 rounds (45.9%)
I need to win 716 of the remaining 1154 rounds (62.0%)


In [5]:
r = mtcrack_floats(draws[-624:])
print(f'MT19937 state cracked successfully.')

MT19937 state cracked successfully.


## Part 2

Ok, so now we know all future outcomes. Let's put a large number of these in an array.

In [6]:
arr = draws[-1:] + [r.random() for _ in range(10000)]

Now, at the beginning of each round we know there's a maximum number of cards you can hit before you go bust. As an example let's say you go bust if you hit three times. Then there are four possible ways to end this round:
1. [stand], can look at dealer's cards to see if results in a win or a loss
2. [hit,stand], can look at dealer's cards to see if results in a win or a loss
3. [hit,hit,stand], can look at dealer's cards to see if results in a win or a loss
4. [hit,hit,hit], which is a guaranteed loss

The greedy algorithm is to just pick one that results in a win, but this may not be optimal in the long run, as it's possible for example that losing here might allow you win the next 10 rounds in a row.

Instead, we can think of this as a graph theory problem where each of those options (four in our example) is an edge to the beginning of the next round, whose weight is either $(1, 0)$ for a win, or $(0, -1)$ for a loss. Then the goal is to determine whether there exists a path which results in a total weight of $(x,y)$ with $x\geq800$ and $x-y=1337$. We can do this properly by keeping a set of Pareto-optimal scores at each node, but for our implementation we've become lazy and just took the lexicographic maximum at each node.

In [7]:
# keep taking cards until you exceed n
def how_many(n, offset):
    total = 0
    while total <= n:
        total += arr[offset]
        offset += 1
    return total >= 1, offset

def get_edges(offset):
    total_so_far = arr[offset]
    while total_so_far < 1:
        dealer = how_many(total_so_far, offset + 1)
        yield (offset, 1), dealer
        offset += 1
        total_so_far += arr[offset]
    yield (offset, 0), (False, offset + 1)
    
def choose(old, new):
    return old if old and (old[0],-old[1]) >= (new[0],-new[1]) else new

leaf = None
dic = { 0: (r1wins, r1rounds, (b'', None)) }
while dic:
    i = min(dic.keys())
    wins, rounds, parent = dic.pop(i)
    for code, (win, dst) in get_edges(i):
        code_bin = b'\n' * (code[0] - i) + b's\n' * code[1]
        next_state = (wins + win, rounds + 1, (code_bin, parent))
        if rounds + 1 == TOTAL_ROUNDS:
            leaf = choose(leaf, next_state)
        else:
            dic[dst] = choose(dic.get(dst), next_state)
    
print(f'We can win up to {leaf[0]} of 1337 rounds.')
assert leaf[0] >= WIN_THRESHOLD, 'Try again.'

We can win up to 827 of 1337 rounds.


Overall, there's a more than 80% chance that everything worked out fine the first time and we've won at least 800 rounds. (If not, then we just rerun the script from the start.) Then we can just trace the path backwards to figure out exactly what your sequence of hits and stands should be.

In [8]:
def get_codes():
    code, parent = leaf[2]
    while parent:
        yield code
        code, parent = parent

sh.send(b''.join(c for c in list(get_codes())[::-1]))

All that's left to do then is to read out the flag!

In [9]:
sh.recvuntil(b'flag: ')
print(sh.recvline(False))
sh.close()

b'SEE{1337_card_counting_24ca335ed1cabbcf}'
[*] Closed connection to fun.chall.seetf.sg port 30001


---
---
---

## <a name="appendix"></a>Appendix

Let $f(x,y)$ be the probability that you land in the interval $[x,x+y]\subseteq[0,1]$. Then

$$f(x,y) = \int_0^x f(x-t,y)\,dt+y,$$

and solving the ODE we get $f(x,y)=f(0,y)e^x$. Plugging in the boundary condition $f(0,y)=y$ then gives $f(x,y) = ye^x$.

Now let $s(x)$ be the probability that you win if you stand at a current total of $x$. This requires the dealer to not land in $[x,1]$, i.e.

$$s(x) = 1 - f(x,1-x) = 1 - (1-x)e^x.$$

We seek a threshold $T$, above which it is better to stand than to hit. We can find this by identifying the point where hitting exactly once makes no difference to the probability of winning. Then

$$s(T) = \int_T^1 s(t)\,dt,$$

or equivalently, $1-(1-T)e^T = (1-T)-e^T(T-2)-e$, which gives us $T\approx 0.5705565$.

**The strategy here is to keep hitting until the total is above roughly 0.57, then stand (or be bust).**

---

Let $h(x)$ be the probability of winning with the above strategy, given a current total of $x$. Combining the behaviour before and after the threshold gives us

$$h(x) = \int_x^T h(t)\,dt + \int_T^1 s(t)\, dt.$$

The final term is simply $s(T)$, and we can again solve the ODE to get $h(x)=h(0)e^{-x}$. Plugging in the boundary condition $h(T)=s(T)$ gives us the probability of winning, i.e. $h(0)=s(T)e^T \approx 0.4249857$.

**Using this strategy gives us a win rate of roughly 42.5%.**

---

Next, we might be interested in the expected number of cards drawn in a single round using this strategy. Let $c(x)$ be the expected number of cards drawn to get a total of at least $x \in [0,1]$. Then

$$c(x) = 1 + \int_0^x c(x-t)\,dt,$$

and yet again we get an ODE that simply resolves to $c(x)=e^x$.

In particular, we expect to have drawn $c(T)$ cards before the end of our turn. The probability that we go bust here is $s(T)$, so that the dealer does not draw any cards. Otherwise, we have a uniform distribution in $[T,1]$ with density $e^T$. Consequently, the expected number of cards drawn in a single round is

$$e^T + \int_T^1 e^T c(x)\,dx = e^T(1+e-e^T) \approx 3.448325.$$

Now, to crack the state of MT19937 we need to have drawn 624 cards, and since we get one for free at the beginning of a round we only need to have played 623 cards worth of full rounds. Using the numbers above, this roughly means that we will expect to have completed 180.7 full rounds and won 76.78 of those. (We can also compute the variances and stuff, but let's not go there.)

**We expect part one to take 181 rounds, with us winning about 77 rounds.**

Good stuff! This means that on average, we only need to win 723 of our remaining 1156 rounds, for a whopping 62.54% win rate. This is something like 14 standard deviations away (using the 0.57 strategy), so we are definitely not hoping to simply get lucky for part 2, and will need a different strategy.